# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-26-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-26-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-27 05:22:55,33.93911,67.709953,50886,2149,40784,7953.0,Afghanistan,130.717104,4.223166
1,NaN,NaN,NaN,Albania,2020-12-27 05:22:55,41.15330,20.168300,55755,1143,31181,23431.0,Albania,1937.417472,2.050040
2,NaN,NaN,NaN,Algeria,2020-12-27 05:22:55,28.03390,1.659600,97857,2722,65505,29630.0,Algeria,223.157748,2.781610
3,NaN,NaN,NaN,Andorra,2020-12-27 05:22:55,42.50630,1.521800,7806,83,7252,471.0,Andorra,10102.892642,1.063285
4,NaN,NaN,NaN,Angola,2020-12-27 05:22:55,-11.20270,17.873900,17149,399,9976,6774.0,Angola,52.178118,2.326666


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,49378,49621,49681,49817,50013,50190,50433,50655,50810,50886
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,51424,52004,52542,53003,53425,53814,54317,54827,55380,55755
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,93933,94371,94781,95203,95659,96069,96549,97007,97441,97857


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2025,2030,2047,2067,2082,2096,2117,2126,2139,2149
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1055,1066,1074,1088,1098,1111,1117,1125,1134,1143
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2640,2647,2659,2666,2675,2687,2696,2705,2716,2722


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38505,38540,38613,39006,39508,39585,39692,40359,40444,40784
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,26898,27426,27831,28121,28752,29249,29799,30276,30790,31181
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,62089,62487,62869,63260,63644,64020,64401,64777,65144,65505


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3570,3647,3698,3741,3780,3841,3889,3942,3990,3999
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,11364,11556,11722,11827,11952,12155,12321,12521,12666,12708
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1309,1318,1330,1336,1336,1363,1383,1390,1396,1398


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,43,44,44,44,44,44,46,46,46,46
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,145,146,147,147,147,149,151,151,151,151
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,30,30,32,32,32,32,32,32,32,32


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-27 05:22:55,32.539527,-86.644082,3999,46,0,3953.0,"Autauga, Alabama, US",7157.815604,1.150288
686,1075.0,Lamar,Alabama,US,2020-12-27 05:22:55,33.779950,-88.096680,1009,15,0,994.0,"Lamar, Alabama, US",7308.946034,1.486620
687,1077.0,Lauderdale,Alabama,US,2020-12-27 05:22:55,34.901719,-87.656247,6039,68,0,5971.0,"Lauderdale, Alabama, US",6512.525747,1.126014


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,18982634,331909,0
India,10187850,147622,9761538
Brazil,7465806,190795,6654189


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,18982634,331909,0,18650725,2020-12-27 05:22:55,37.936303,-91.379001
India,10187850,147622,9761538,278690,2020-12-27 05:22:55,23.088275,81.806127
Brazil,7465806,190795,6654189,620822,2020-12-27 05:22:55,-12.669522,-48.480493
Russia,2992123,53539,2402122,536462,2020-12-27 05:22:55,54.546312,62.120860
France,2607688,62694,195638,2349356,2020-12-27 05:22:55,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2120610,24218,0
Texas,1668138,26916,0
Florida,1264588,21135,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2120610,24218,0,2096392,2020-12-27 05:22:55,37.843962,-120.728594
Texas,1668138,26916,0,1641222,2020-12-27 05:22:55,31.660643,-98.653069
Florida,1264588,21135,0,1243453,2020-12-27 05:22:55,28.940755,-82.700744
Illinois,934142,17224,0,916918,2020-12-27 05:22:55,39.843618,-89.178330
New York,914522,37286,0,877236,2020-12-27 05:22:55,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,706448,9438,0
Illinois,Cook,383480,8060,0
Arizona,Maricopa,302640,4906,0
Florida,Miami-Dade,286662,4127,0
Texas,Harris,225443,3328,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,706448,9438,0,697010,2020-12-27 05:22:55,34.308284,-118.228241,6037.0
Illinois,Cook,383480,8060,0,375420,2020-12-27 05:22:55,41.841448,-87.816588,17031.0
Arizona,Maricopa,302640,4906,0,297734,2020-12-27 05:22:55,33.348359,-112.491815,4013.0
Florida,Miami-Dade,286662,4127,0,282535,2020-12-27 05:22:55,25.611236,-80.551706,12086.0
Texas,Harris,225443,3328,0,222115,2020-12-27 05:22:55,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,50655,54827,97007,7699,17029,154,1563865,156142,28274,347204,...,2195144,15457,76456,1,111245,1432,129080,2092,19234,12786
2020-12-25,50810,55380,97441,7756,17099,155,1574554,156763,28297,349055,...,2227947,15848,76568,1,111603,1439,130598,2092,19571,12880
2020-12-26,50886,55755,97857,7806,17149,155,1578267,157349,28308,350484,...,2262735,16218,76666,1,111951,1440,131904,2092,19671,12963


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,2126,1125,2705,83,393,5,42314,2714,908,5745,...,69732,135,612,0,1006,35,1243,606,381,339
2020-12-25,2139,1134,2716,83,396,5,42422,2733,908,5783,...,70302,143,612,0,1010,35,1260,606,381,341
2020-12-26,2149,1143,2722,83,399,5,42501,2752,908,5843,...,70513,147,612,0,1013,35,1281,606,382,341


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,40359,30276,64777,7171,9729,146,1384277,136766,25724,315952,...,4821,10040,73746,1,105479,1281,105233,1384,18036,10314
2020-12-25,40444,30790,65144,7203,9921,146,1384277,137815,25727,318154,...,4822,10479,73911,1,105804,1303,106718,1384,18127,10468
2020-12-26,40784,31181,65505,7252,9976,146,1402227,138669,25730,320753,...,4910,10847,74291,1,106105,1303,108118,1384,18200,10519


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3570,3647,3698,3741,3780,3841,3889,3942,3990,3999
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,912,925,941,946,966,979,986,997,1007,1009
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,5346,5428,5531,5582,5644,5722,5859,5996,6048,6039
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1672,1708,1739,1757,1783,1819,1859,1907,1921,1924
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,9055,9188,9283,9337,9372,9490,9609,9715,9779,9800


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,338801,45104,0,480319,211145,2040188,319530,172743,53015,49,...,546497,1659409,260589,6781,1967,323915,233093,77239,503706,42664
2020-12-25,342426,45104,0,486935,213267,2060797,322189,172743,53653,49,...,546497,1662724,260589,6781,1979,327993,233093,78836,505345,42664
2020-12-26,343458,45479,0,493041,213969,2120610,323619,172743,54473,49,...,560892,1668138,264078,6966,1979,329577,236719,80177,506022,43146


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-24        3942   12521    1390  1711   4405     770   1434    9071   
2020-12-25        3990   12666    1396  1725   4441     777   1448    9167   
2020-12-26        3999   12708    1398  1739   4446     825   1446    9198   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-24         2191     1343  ...    1860    418     2432      579   
2020-12-25         2200     1368  ...    1860    418     2432      579   
2020-12-26         2203     1368  ...    1898    420     2458      582   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-24           2820  2039  1475          0      726    460  
2020-12-25           2820  2039  1475          0      726    460  
2020-12-26           2845  2080  1495          0      751    461  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,4676,199,0,8294,3406,23881,4550,5791,888,0,...,6431,26707,1204,120,23,4791,3184,1228,5019,373
2020-12-25,4680,199,0,8409,3438,23958,4586,5791,891,0,...,6431,26896,1204,120,23,4820,3184,1247,5025,373
2020-12-26,4685,200,0,8424,3441,24218,4592,5791,895,0,...,6443,26916,1212,121,23,4840,3184,1253,5029,373


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-24          46     151      32   42     58      20     45     148   
2020-12-25          46     151      32   42     58      20     45     149   
2020-12-26          46     151      32   42     58      20     45     149   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-24           61       23  ...       7      8       15        5   
2020-12-25           61       23  ...       7      8       15        5   
2020-12-26           61       23  ...       7      8       15        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-24             15     2     7          0       12      2  
2020-12-25             15     2     7          0       12      2  
2020-12-26             15     2     7          0       12      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.004402,0.009389,0.004744,0.003912,0.005788,0.000000,0.000000,0.004516,0.000425,0.008268,...,0.018158,0.050782,0.001848,0.0,0.001991,0.007741,0.013378,0.002396,0.005857,0.010272
2020-12-25,0.003060,0.010086,0.004474,0.007404,0.004111,0.006494,0.006835,0.003977,0.000813,0.005331,...,0.014943,0.025296,0.001465,0.0,0.003218,0.004888,0.011760,0.000000,0.017521,0.007352
2020-12-26,0.001496,0.006771,0.004269,0.006447,0.002924,0.000000,0.002358,0.003738,0.000389,0.004094,...,0.015614,0.023347,0.001280,0.0,0.003118,0.000695,0.010000,0.000000,0.005110,0.006444


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.004251,0.007162,0.003338,0.012195,0.000000,0.0,0.000000,0.008547,0.0,0.016095,...,0.008314,0.054688,0.0,NaN,0.003992,0.0,0.013866,0.0,0.002632,0.027273
2020-12-25,0.006115,0.008000,0.004067,0.000000,0.007634,0.0,0.002552,0.007001,0.0,0.006614,...,0.008174,0.059259,0.0,NaN,0.003976,0.0,0.013677,0.0,0.000000,0.005900
2020-12-26,0.004675,0.007937,0.002209,0.000000,0.007576,0.0,0.001862,0.006952,0.0,0.010375,...,0.003001,0.027972,0.0,NaN,0.002970,0.0,0.016667,0.0,0.002625,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.016804,0.016007,0.005838,0.009147,0.000000,0.0,0.000000,0.008316,0.000078,0.009686,...,0.020750,0.048673,0.002133,0.0,0.002185,0.000000,0.022255,0.0,0.003729,0.005361
2020-12-25,0.002106,0.016977,0.005666,0.004462,0.019735,0.0,0.000000,0.007670,0.000117,0.006969,...,0.000207,0.043725,0.002237,0.0,0.003081,0.017174,0.014112,0.0,0.005045,0.014931
2020-12-26,0.008407,0.012699,0.005542,0.006803,0.005544,0.0,0.012967,0.006197,0.000117,0.008169,...,0.018250,0.035118,0.005141,0.0,0.002845,0.000000,0.013119,0.0,0.004027,0.004872


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,0.012649,0.006157,NaN,0.014888,0.015408,0.018824,0.009573,0.011939,0.014933,0.0,...,0.009713,0.006575,0.011222,0.015120,0.008718,0.014984,0.012559,0.017159,0.006520,0.000000
2020-12-25,0.010699,0.000000,NaN,0.013774,0.010050,0.010102,0.008322,0.000000,0.012034,0.0,...,0.000000,0.001998,0.000000,0.000000,0.006101,0.012590,0.000000,0.020676,0.003254,0.000000
2020-12-26,0.003014,0.008314,NaN,0.012540,0.003292,0.029024,0.004438,0.000000,0.015283,0.0,...,0.026340,0.003256,0.013389,0.027282,0.000000,0.004829,0.015556,0.017010,0.001340,0.011298


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-24      0.013628  0.016232  0.005061  0.016637  0.008702  0.006536   
2020-12-25      0.012177  0.011581  0.004317  0.008182  0.008173  0.009091   
2020-12-26      0.002256  0.003316  0.001433  0.008116  0.001126  0.061776   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2020-12-24      0.007730  0.011485  0.006893  0.010534  ...  0.00000   
2020-12-25      0.009763  0.010583  0.004108  0.018615  ...  0.00000   
2020-12-26     -0.001381  0.003382  0.001364  0.000000  ...  0.02043   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-24      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2020-12-25      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2020-12-26      0.004785  0.010691  0.005181   0.008865  0.020108  0.013559   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-24            NaN  0.000000  0.000000  
2020-12-25            NaN  0.000000  0.000000  
2020-12-26            NaN  0.034435  0.002174  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,0.019403,0.010152,NaN,0.014060,0.008886,0.013711,0.019722,0.009589,0.018349,NaN,...,0.007994,0.011399,0.006689,0.025641,0.0,0.006513,0.006958,0.028476,0.013325,0.0
2020-12-25,0.000855,0.000000,NaN,0.013865,0.009395,0.003224,0.007912,0.000000,0.003378,NaN,...,0.000000,0.007077,0.000000,0.000000,0.0,0.006053,0.000000,0.015472,0.001195,0.0
2020-12-26,0.001068,0.005025,NaN,0.001784,0.000873,0.010852,0.001308,0.000000,0.004489,NaN,...,0.001866,0.000744,0.006645,0.008333,0.0,0.004149,0.000000,0.004812,0.000796,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                \
Admin2         Autauga Baldwin Barbour Bibb    Blount Bullock Butler   
2020-12-24         0.0     0.0     0.0  0.0  0.017544     0.0    0.0   
2020-12-25         0.0     0.0     0.0  0.0  0.000000     0.0    0.0   
2020-12-26         0.0     0.0     0.0  0.0  0.000000     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-12-24      0.020690      0.0      0.0  ...     0.0    0.0      0.0   
2020-12-25      0.006757      0.0      0.0  ...     0.0    0.0      0.0   
2020-12-26      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-24          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-25          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-26          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.004277,0.009080,0.004737,0.004095,0.006005,0.001932,0.002489,0.004471,0.000630,0.007021,...,0.017983,0.047694,0.001783,0.0,0.002228,0.004704,0.013496,0.001202,0.007296,0.009888
2020-12-25,0.003668,0.009583,0.004605,0.005750,0.005058,0.004213,0.004662,0.004224,0.000722,0.006176,...,0.016463,0.036495,0.001624,0.0,0.002723,0.004796,0.012628,0.000601,0.012409,0.008620
2020-12-26,0.002582,0.008177,0.004437,0.006098,0.003991,0.002106,0.003510,0.003981,0.000555,0.005135,...,0.016039,0.029921,0.001452,0.0,0.002921,0.002745,0.011314,0.000300,0.008759,0.007532


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.006439,0.007695,0.003481,0.008133,0.001723,0.000061,0.001525,0.007785,1.642906e-11,0.018186,...,0.008763,0.047693,1.001086e-07,NaN,0.004350,3.829739e-36,0.018709,-4.145830e-07,0.003738,0.019031
2020-12-25,0.006277,0.007848,0.003774,0.004067,0.004678,0.000031,0.002038,0.007393,8.214532e-12,0.012400,...,0.008468,0.053476,5.005429e-08,NaN,0.004163,1.914869e-36,0.016193,-2.072915e-07,0.001869,0.012465
2020-12-26,0.005476,0.007892,0.002991,0.002033,0.006127,0.000015,0.001950,0.007172,4.107266e-12,0.011388,...,0.005735,0.040724,2.502714e-08,NaN,0.003567,9.574347e-37,0.016430,-1.036458e-07,0.002247,0.006233


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-24,0.010494,0.017141,0.005916,0.007192,0.001796,0.004833,0.001834,0.008118,0.000115,0.010254,...,0.029791,0.048093,0.001913,0.0,0.002517,0.001280,0.019427,-4.528356e-07,0.003759,0.006673
2020-12-25,0.006300,0.017059,0.005791,0.005827,0.010765,0.002416,0.000917,0.007894,0.000116,0.008612,...,0.014999,0.045909,0.002075,0.0,0.002799,0.009227,0.016769,-2.264178e-07,0.004402,0.010802
2020-12-26,0.007353,0.014879,0.005666,0.006315,0.008154,0.001208,0.006942,0.007045,0.000116,0.008390,...,0.016624,0.040513,0.003608,0.0,0.002822,0.004613,0.014944,-1.132089e-07,0.004215,0.007837


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,0.013009,0.007022,NaN,0.014158,0.013552,0.020085,0.009130,0.011751,0.013177,2.190450e-86,...,0.011793,0.009312,0.010344,0.013594,0.007752,0.014232,0.010990,0.016971,0.006165,0.003524
2020-12-25,0.011854,0.003511,NaN,0.013966,0.011801,0.015093,0.008726,0.005876,0.012605,1.095225e-86,...,0.005896,0.005655,0.005172,0.006797,0.006927,0.013411,0.005495,0.018824,0.004710,0.001762
2020-12-26,0.007434,0.005913,NaN,0.013253,0.007546,0.022059,0.006582,0.002938,0.013944,5.476125e-87,...,0.016118,0.004455,0.009280,0.017040,0.003463,0.009120,0.010525,0.017917,0.003025,0.006530


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-24      0.013511  0.015056  0.009124  0.015578  0.010838  0.006235   
2020-12-25      0.012844  0.013318  0.006720  0.011880  0.009505  0.007663   
2020-12-26      0.007550  0.008317  0.004076  0.009998  0.005316  0.034719   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-24      0.007125  0.012397  0.007498  0.011022  ...  0.006095   
2020-12-25      0.008444  0.011490  0.005803  0.014819  ...  0.003048   
2020-12-26      0.003531  0.007436  0.003583  0.007409  ...  0.011739   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-24      0.003486  0.001632  0.004014   0.005953  0.002437  0.005630   
2020-12-25      0.001743  0.000816  0.002007   0.002977  0.001219  0.002815   
2020-12-26      0.003264  0.005753  0.003594   0.005921  0.010663  0.008187   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-24           -1.0  0.005212  0.005461  
2020-12-25           -1.0  0.002606  0.002730  
2020-12-26           -1.0  0.018521  0.002452  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-24,0.019586,0.015919,NaN,0.011730,0.010926,0.013412,0.015772,0.007999,0.010359,NaN,...,0.012705,0.010038,0.010426,0.026421,3.075945e-17,0.007950,0.007132,0.024829,0.014607,0.008135
2020-12-25,0.010221,0.007960,NaN,0.012798,0.010160,0.008318,0.011842,0.004000,0.006869,NaN,...,0.006352,0.008557,0.005213,0.013210,1.537972e-17,0.007002,0.003566,0.020151,0.007901,0.004068
2020-12-26,0.005645,0.006492,NaN,0.007291,0.005516,0.009585,0.006575,0.002000,0.005679,NaN,...,0.004109,0.004650,0.005929,0.010772,7.689862e-18,0.005575,0.001783,0.012481,0.004349,0.002034


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-12-24      0.011590  0.005273  0.001042  0.000127  0.023500   
2020-12-25      0.005795  0.002636  0.000521  0.000063  0.011750   
2020-12-26      0.002898  0.001318  0.000261  0.000032  0.005875   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-12-24      2.509670e-08  0.000180  0.018842  0.002695 -0.000326  ...   
2020-12-25      1.254835e-08  0.000090  0.012799  0.001348 -0.000163  ...   
2020-12-26      6.274174e-09  0.000045  0.006400  0.000674 -0.000081  ...   

Province_State       Wyoming                                               \
Admin2                  Park        Platte  Sheridan  Sublette Sweetwater   
2020-12-24      6.358994e-07  8.716850e-09  0.000603  0.000022   0.000078   
2020-12-25      3.179497e-07  4.358425e-09  0.000301  0.000011   0.000039   
2020-12-26      1.589749e-07  2.179212e-09  0.000151  0.000006   0.000019   

Province_State                                                                 
Admin2                 Teton         Uinta Unassigned  Washakie        Weston  
2020-12-24      2.220446e-16  3.365676e-06  -0.501961  0.011366  5.960464e-08  
2020-12-25      1.110223e-16  1.682838e-06  -0.501961  0.005683  2.980232e-08  
2020-12-26      5.551115e-17  8.414189e-07  -0.501961  0.002842  1.490116e-08  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201226,AK,43990,NaN,1201073,NaN,totalTestsViral,1245063,79.0,991.0,...,1245063,1,8,35472bedc43830797ee07bc6805144d9f7841037,0,0,0,0,0,NaN
1,20201226,AL,343458,64801.0,1560978,NaN,totalTestsPeopleViral,1839635,2470.0,32115.0,...,1904436,5,161,ddecf6729323cb58b780c99afd14bad0af8e82ef,0,0,0,0,0,NaN
2,20201226,AR,213969,34329.0,1827152,NaN,totalTestsViral,2006792,1059.0,10956.0,...,2041121,3,29,c6062c26be40d4c9484557e3ce7de3bf2f266899,0,0,0,0,0,NaN
3,20201226,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,ce8942baf31c8aeb6f83fabc34830220b5458c6f,0,0,0,0,0,NaN
4,20201226,AZ,493041,22567.0,2293262,NaN,totalTestsViral,4893296,4165.0,35337.0,...,2786303,15,360,386ffa6fe831df73dd18f351fee2e1a4ac294228,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201226,AK,43990.0,NaN,1201073.0,NaN,totalTestsViral,1245063.0,79.0,991.0,...,1245063,1,8,35472bedc43830797ee07bc6805144d9f7841037,0,0,0,0,0,NaN
1,20201226,AL,343458.0,64801.0,1560978.0,NaN,totalTestsPeopleViral,1839635.0,2470.0,32115.0,...,1904436,5,161,ddecf6729323cb58b780c99afd14bad0af8e82ef,0,0,0,0,0,NaN
2,20201226,AR,213969.0,34329.0,1827152.0,NaN,totalTestsViral,2006792.0,1059.0,10956.0,...,2041121,3,29,c6062c26be40d4c9484557e3ce7de3bf2f266899,0,0,0,0,0,NaN
3,20201226,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,ce8942baf31c8aeb6f83fabc34830220b5458c6f,0,0,0,0,0,NaN
4,20201226,AZ,493041.0,22567.0,2293262.0,NaN,totalTestsViral,4893296.0,4165.0,35337.0,...,2786303,15,360,386ffa6fe831df73dd18f351fee2e1a4ac294228,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-26,AK,43990,NaN,1201073,NaN,totalTestsViral,1245063,79.0,991.0,NaN,...,1245063,1,8,35472bedc43830797ee07bc6805144d9f7841037,0,0,0,0,0,NaN
2020-12-26,AL,343458,64801.0,1560978,NaN,totalTestsPeopleViral,1839635,2470.0,32115.0,NaN,...,1904436,5,161,ddecf6729323cb58b780c99afd14bad0af8e82ef,0,0,0,0,0,NaN
2020-12-26,AR,213969,34329.0,1827152,NaN,totalTestsViral,2006792,1059.0,10956.0,364.0,...,2041121,3,29,c6062c26be40d4c9484557e3ce7de3bf2f266899,0,0,0,0,0,NaN
2020-12-26,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,ce8942baf31c8aeb6f83fabc34830220b5458c6f,0,0,0,0,0,NaN
2020-12-26,AZ,493041,22567.0,2293262,NaN,totalTestsViral,4893296,4165.0,35337.0,983.0,...,2786303,15,360,386ffa6fe831df73dd18f351fee2e1a4ac294228,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-26,AK,43990.0,NaN,1201073.0,NaN,totalTestsViral,1245063.0,79.0,991.0,NaN,...,1245063,1,8,35472bedc43830797ee07bc6805144d9f7841037,0,0,0,0,0,NaN
2020-12-26,AL,343458.0,64801.0,1560978.0,NaN,totalTestsPeopleViral,1839635.0,2470.0,32115.0,NaN,...,1904436,5,161,ddecf6729323cb58b780c99afd14bad0af8e82ef,0,0,0,0,0,NaN
2020-12-26,AR,213969.0,34329.0,1827152.0,NaN,totalTestsViral,2006792.0,1059.0,10956.0,364.0,...,2041121,3,29,c6062c26be40d4c9484557e3ce7de3bf2f266899,0,0,0,0,0,NaN
2020-12-26,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,ce8942baf31c8aeb6f83fabc34830220b5458c6f,0,0,0,0,0,NaN
2020-12-26,AZ,493041.0,22567.0,2293262.0,NaN,totalTestsViral,4893296.0,4165.0,35337.0,983.0,...,2786303,15,360,386ffa6fe831df73dd18f351fee2e1a4ac294228,0,0,0,0,0,NaN
